In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
pwd

'C:\\Users\\koros\\Перенос с облака\\Алгоритмы анализа данных\\kaggle_regression'

In [7]:
Train = pd.read_csv("train.csv")
Train.describe()

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,4999.50000,45.878000,1.986800,1699.105000,1.719500,0.375000,0.132900,0.109600,0.053700,0.032100,0.019400,63.737300
std,2886.89568,8.043929,1.772213,524.886654,0.792264,0.484147,0.339484,0.312406,0.225436,0.176274,0.137933,14.705574
min,0.00000,23.000000,0.000000,200.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,32.000000
25%,2499.75000,40.000000,0.000000,1300.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,53.000000
50%,4999.50000,46.000000,2.000000,1500.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,63.000000
75%,7499.25000,51.000000,3.000000,2150.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,74.000000
max,9999.00000,68.000000,10.000000,3950.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,100.000000


In [8]:
columns_list = Train.columns.tolist()
y_name = columns_list.pop()
y = Train[[y_name]]
y

,mean_exam_points
0,61
1,76
2,53
3,54
4,57
...,...
9995,83
9996,58
9997,62
9998,56


In [9]:
X = Train[columns_list]
X.head()

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history
0,0,40.0,0.0,1400.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1,48.0,4.0,2850.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,39.0,0.0,1200.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,46.0,5.0,1400.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,43.0,1.0,1500.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


По данным kaggle в этом датасете нет пропусков. 
С 5 по 10 столбцы (название предмета) - категориальные, мы их отбросим в анализе, так же как и 0 (Id).
Вполне возможно, что преподаваемый предмет коррелирует с целевой переменной, но вряд ли анализ много потеряет, если мы эти столбцы уберем.

In [10]:
X_ = X.drop(['Id', 'physics', 'chemistry', 'biology', 'english', 'geography', 'history'], axis=1)
X_.head()

,age,years_of_experience,lesson_price,qualification
0,40.0,0.0,1400.0,1.0
1,48.0,4.0,2850.0,3.0
2,39.0,0.0,1200.0,1.0
3,46.0,5.0,1400.0,1.0
4,43.0,1.0,1500.0,1.0


Остальные (кроме последнего целевого) нормируем.

In [11]:
columns_list = X_.columns.tolist()
columns_list

['age', 'years_of_experience', 'lesson_price', 'qualification']

In [12]:
for column in columns_list:
    min_el = min(X_[column])
    max_el = max(X_[column])
    for i in range(X_.shape[0]):
        X_[column][i] = (X_[column][i] - min_el) / (max_el - min_el)
X_.head()

,age,years_of_experience,lesson_price,qualification
0,0.377778,0.0,0.320000,0.000000
1,0.555556,0.4,0.706667,0.666667
2,0.355556,0.0,0.266667,0.000000
3,0.511111,0.5,0.320000,0.000000
4,0.444444,0.1,0.346667,0.000000


Проанализируем датасет методом стохастического градиентного спуска

In [14]:
#Среднеквадратичная ошибка
def mserror(X, w, y_pred):
    y = X.dot(w)
    return (sum((y - y_pred)**2)) / len(y)

In [104]:
x_nmp_ = X_.to_numpy()
y_nmp_ = y.to_numpy()
display(y_nmp_)

array([[61],
       [76],
       [53],
       ...,
       [62],
       [56],
       [52]], dtype=int64)

In [108]:
y_nmp_.transpose()
y_nmp = y_nmp_.transpose()[0]
y_nmp

array([61, 76, 53, ..., 62, 56, 52], dtype=int64)

In [109]:
print(f'Корреляция 0 признака с целевой переменной {np.corrcoef(x_nmp_[:, 0], y_nmp)[0][1]}')
print(f'Корреляция 1 признака с целевой переменной {np.corrcoef(x_nmp_[:, 1], y_nmp)[0][1]}')
print(f'Корреляция 2 признака с целевой переменной {np.corrcoef(x_nmp_[:, 2], y_nmp)[0][1]}')
print(f'Корреляция 3 признака с целевой переменной {np.corrcoef(x_nmp_[:, 3], y_nmp)[0][1]}')

Корреляция 0 признака с целевой переменной -0.007143712055381954
Корреляция 1 признака с целевой переменной 0.18459909784601533
Корреляция 2 признака с целевой переменной 0.6640610251757544
Корреляция 3 признака с целевой переменной 0.6982607436001597


In [94]:
def get_max_idxs(s):
    #Возвращает индексы максимальных элементов nd.array s
    s_sort = s.tolist()
    s_sort.sort(reverse=True)
    s_l = s.tolist()
    idxs = [s_l.index(s_sort[0]), s_l.index(s_sort[1])]
    return idxs

In [95]:
def pca_svd(X):
    X_ = np.dot(X.T,X)
    U, s, V_T = np.linalg.svd(X_)
    V = V_T.T
    max_idx = get_max_idxs(s)
    max_idx.sort()
    max_idx_tu = tuple(max_idx)
    W = V[:, max_idx_tu]
    Z = np.dot(X,W)
    return Z

In [110]:
x_nmp = pca_svd(x_nmp_)
x_nmp

array([[-0.4347939 ,  0.16360976],
       [-1.13897256, -0.32563259],
       [-0.39024854,  0.16014992],
       ...,
       [-0.34968925,  0.09373934],
       [-0.3689888 ,  0.08970428],
       [-0.36597667,  0.09629597]])

In [112]:
%%time
# инициализируем начальный вектор весов
w = np.zeros(x_nmp.shape[1])

# список векторов весов после каждой итерации
w_list = [w.copy()]

# список значений ошибок после каждой итерации
errors = []

# шаг градиентного спуска
eta = 0.01

# максимальное число итераций
max_iter = 1e3

# критерий сходимости (разница весов, при которой алгоритм останавливается)
min_weight_dist = 1e-8

# зададим начальную разницу весов большим числом
weight_dist = np.inf

# счетчик итераций
iter_num = 0

np.random.seed(1234)

# ход градиентного спуска
while weight_dist > min_weight_dist and iter_num < max_iter:
    
    # генерируем случайный индекс объекта выборки
    train_ind = np.random.randint(x_nmp.shape[0], size=1)
    
    y_pred = np.dot(x_nmp[train_ind], w)
    new_w = w - eta * 2 / y_nmp[train_ind].shape[0] * np.dot(x_nmp[train_ind].T, y_pred - y_nmp[train_ind])
    #Почему-то новые индексы генерируются в матрицу (4,4) поэтому приходится срезать

    #new_w = long_new_w[:,0]
    #new_w

    weight_dist = np.linalg.norm(new_w - w, ord=2)
 
    error = mserror(x_nmp, new_w, y_nmp)
    
    w_list.append(new_w.copy())
    errors.append(error)
    
    #if iter_num % 100 == 0:
    #    print(f'Iteration #{iter_num}: W_new = {new_w}, MSE = {round(error,2)}')

    iter_num += 1
    w = new_w
    
w_list = np.array(w_list)

print(f'Iter {iter_num}: error - {error}, weights: {new_w}')
print(f'В случае использования стохастического градиентного спуска ошибка составляет {round(errors[-1], 4)}')

Iter 1000: error - 267.6492972407381, weights: [-81.69631497   0.40892929]
В случае использования стохастического градиентного спуска ошибка составляет 267.6493
Wall time: 2.86 s


In [1]:
X_test = test_df.to_numpy()
predictions_result = gb_predict(X_test, trees, eta)

submit = pd.read_csv('/kaggle/input/gb-tutors-expected-math-exam-results/sample_submission.csv')
submit['mean_exam_points'] = predictions_result
submit.to_csv('rf1_submit.csv', index=False)
submit

NameError: name 'y_pred' is not defined